# <font color = 'indianred'> **1. Setting up the Environment** </font>



In [ ]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output
  !pip install torchtext -qq
  !pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
  basepath = '/content/drive/MyDrive/data/'
else:
  basepath = '/home/harpreet/Insync/google_drive_shaannorr/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<font color = 'indianred'> *Load Libraries* </font>

In [ ]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

# <font color = 'indianred'> **2. Load Data set**
    


In [ ]:
base_folder = Path(basepath)
data_folder = base_folder / 'datasets/my_project'
model_folder = base_folder / 'models/nlp_spring_2024/my_project'

In [ ]:
model_folder.mkdir(exist_ok=True, parents=True)
data_folder.mkdir(exist_ok=True, parents=True)

In [ ]:

# Load train data
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/data/datasets/train.csv")


# <font color = 'indianred'> **3. Accessing and Manuplating Splits**</font>



<font color = 'indianred'>*Subsetting the data*

In [ ]:
# Split features and labels
y = train_data[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love','optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values.astype(float)
X = train_data.drop(columns=['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love','optimism', 'pessimism', 'sadness', 'surprise', 'trust'])

# Split data into train, test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=1)

# Further split train data into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
# Create a subset of the training data
subset_size = 1000  # Define the size of the subset
X_train_subset = X_train[:subset_size]
y_train_subset = y_train[:subset_size]

<font color = 'indianred'>*Extract Splits*

In [ ]:
# Convert y_train and y_valid to torch LongTensor
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

In [ ]:
# Convert X_train, X_valid to lists
X_train_list = list(X_train['Tweet'])
X_valid_list = list(X_valid['Tweet'])
X_test_list = list(X_test['Tweet'])

<font color = 'indianred'>*Create futher subdivions of the splits*</font>

In [ ]:
# Create Datasets
train_set = Dataset.from_dict({
    'texts': X_train_list,
    'labels': y_train
})

valid_set = Dataset.from_dict({
    'texts': X_valid_list,
    'labels': y_valid
})

test_set = Dataset.from_dict({
    'texts': X_test_list,
    'labels': y_test
})

In [ ]:
train_set

Dataset({
    features: ['texts', 'labels'],
    num_rows: 289
})

In [ ]:


# Print the shapes of train, validation, and test sets
print("Train set shape:", len(train_set))
print("Validation set shape:", len(valid_set))
print("Test set shape:", len(test_set))


Train set shape: 289
Validation set shape: 97
Test set shape: 7338


In [ ]:

# Create DatasetDict
from datasets import DatasetDict

data_dict = DatasetDict({
    "train": train_set,
    "valid": valid_set
})

In [ ]:

# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
import nltk
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# <font color = 'indianred'>**4. Load pre-trained Tokenizer**</font>



##  <font color = 'indianred'> **4.1 Download pre-trained model**

In [ ]:
# Instantiate the T5 tokenizer
checkpoint = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

#<font color = 'indianred'> **5. Create function for Tokenizer**



In [ ]:
# Define the prefix for the task
prefix = "multi-label classification: "

# Define the preprocess function
def preprocess_function(examples):
    # Add the prefix to the texts
    inputs = [prefix + text for text in examples["texts"]]

    # Tokenize the input texts
    inputs_encodings = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)

    # Prepare the labels
    labels = []
    for label_list in examples["labels"]:
        label_str = " ".join([class_names[i] for i, label in enumerate(label_list) if label == 1])
        labels.append(label_str)

    # Tokenize the labels
    labels_encodings = tokenizer(labels, padding="max_length", truncation=True, max_length=128)

    # Update the inputs dictionary with the labels
    inputs_encodings["labels"] = labels_encodings["input_ids"]

    return inputs_encodings


<font color = 'indianred'> *Use map function to apply tokenization to all splits*

In [ ]:
# Define the class names
class_names = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Map the preprocess function to your dataset
tokenized_dataset = data_dict.map(preprocess_function, batched=True)

# Remove the original 'texts' column
tokenized_dataset = tokenized_dataset.remove_columns(['texts'])

# Set the dataset format to 'torch'
tokenized_dataset.set_format(type='torch')


Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [ ]:
data_dict['train'][0]

{'texts': 'Theme of week: Ask the Lord for strength &amp; perspective to persevere in #integrity and effort, despite being #disheartened &amp; disappointed.',
 'labels': [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]}

#  <font color = 'indianred'> **6. Model Training**

##  <font color = 'indianred'> **6.1 Downaload and Modify Model Config File**

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import T5Config

# Load the configuration
config = T5Config.from_pretrained("google/flan-t5-base", num_labels=len(class_names))

# Get class names from the dataset
class_names = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Create id to label and label to id mappings
id2label = {id_: label_ for id_, label_ in enumerate(class_names)}
label2id = {label_: id_ for id_, label_ in enumerate(class_names)}

# Update the configuration with mappings
config.id2label = id2label
config.label2id = label2id

# Associate the updated configuration with the model
model.config = config
config

T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_

##  <font color = 'indianred'> **6.2 compute_metrics function** </font>



In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, hamming_loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #print(eval_pred.label_ids)

    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert logits to a tensor if it's not already
    if not isinstance(logits, torch.Tensor):
        logits = torch.tensor(logits)

    # Print the shapes to confirm they are correct
    #print(f'Logits shape: {logits.shape}, Labels shape: {labels.shape}')

    # Ensure that labels are also a tensor
    if not isinstance(labels, torch.Tensor):
        labels = torch.tensor(labels)

    # Apply sigmoid to logits to get probabilities
    probabilities = torch.sigmoid(logits).numpy()

    # Choose a threshold to convert probabilities to binary predictions
    threshold = 0.5
    predictions = (probabilities > threshold).astype(int)
    predictions = predictions[:, :, 0]
    # Flatten the arrays to compare all labels and predictions
    predictions = predictions.flatten()
    labels = labels.flatten()

    # Compute metrics using sklearn
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    hamming_loss_metric = hamming_loss(labels, predictions)

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hamming_loss': hamming_loss_metric
    }


In [ ]:
#!pip install accelerate>=0.21.0
#!pip install transformers[torch]

## <font color = 'indianred'> **6.3 Training Arguments**</font>







In [ ]:
from pathlib import Path
from transformers import TrainingArguments, Trainer

# Define the directory where model checkpoints will be saved
run_name = "flan_T5_base_final"
base_folder = Path("/content/drive/MyDrive/data")
model_folder = base_folder / "models" / run_name
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    learning_rate=3e-5,
    optim='adamw_torch',
    output_dir=str(model_folder),
    evaluation_strategy='steps',
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    load_best_model_at_end=True,
    save_total_limit=10,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=10,
    report_to='wandb',
    run_name=run_name,
    fp16=True,
    max_grad_norm=1.0
)




##  <font color = 'indianred'> **6.4 Initialize Trainer**</font>



In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
tokenized_dataset["train"][1]

{'labels': tensor([ 3922,   333, 24543,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0

## <font color = 'indianred'> **6.5 Setup WandB**</font>

In [ ]:
wandb.login()
%env WANDB_PROJECT = homework_6_sentiment_analysis

wandb: Currently logged in as: likith-gv. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=homework_6_sentiment_analysis


##  <font color = 'indianred'> **6.6 Training and Validation**

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
10,0.000000,nan,0.972213,0.061619,0.998263,0.062500,0.027787,7.689200,12.615000,0.910000
20,0.000000,nan,0.972213,0.061619,0.998263,0.062500,0.027787,9.416600,10.301000,0.743000
30,0.000000,nan,0.972213,0.061619,0.998263,0.062500,0.027787,9.359200,10.364000,0.748000
40,0.000000,nan,0.972213,0.061619,0.998263,0.062500,0.027787,13.641700,7.111000,0.513000
50,0.000000,nan,0.972213,0.061619,0.998263,0.062500,0.027787,8.443000,11.489000,0.829000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=57, training_loss=0.0, metrics={'train_runtime': 111.0323, 'train_samples_per_second': 7.809, 'train_steps_per_second': 0.513, 'total_flos': 148421156143104.0, 'train_loss': 0.0, 'epoch': 3.0})

<font color = 'indianred'> *Evaluate model on Validation Set* </font>


In [ ]:
eval_results = trainer.evaluate(tokenized_dataset["valid"])

In [ ]:
eval_results

{'eval_loss': nan,
 'eval_accuracy': 0.9722132731958762,
 'eval_f1': 0.06161943071834035,
 'eval_precision': 0.9982633295747423,
 'eval_recall': 0.0625,
 'eval_hamming_loss': 0.02778672680412371,
 'eval_runtime': 11.1193,
 'eval_samples_per_second': 8.724,
 'eval_steps_per_second': 0.63,
 'epoch': 3.0}

In [ ]:
wandb.log({"eval_accuracy": eval_results["eval_accuracy"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1"]})

In [ ]:
valid_output = trainer.predict(tokenized_dataset["valid"])

<font color = 'indianred'> *Get best checkpoint*</font>


In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")


The best model was saved at step 10.


#  <font color = 'indianred'> **7. Performance on Test Set** </font>
For test set evaluation, we will perfrom following steps:



In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
checkpoint

'/content/drive/MyDrive/data/models/flan_T5_base_final/checkpoint-10'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/data/models/flan_T5_base_final/checkpoint-10 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


<font color = 'indianred'>*Preprocess function for Test Set*</font>

In [ ]:
# Define the preprocess function for test data
def preprocess_function_test(examples):
    # Add the prefix to the texts
    inputs = [prefix + text for text in examples["texts"]]

    # Tokenize the input texts
    inputs_encodings = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)

    return inputs_encodings


In [ ]:
# Select a smaller subset of the test data
subset_size_test = 1  # Define the size of the subset
X_test_subset = X_test[:subset_size_test]
y_test_subset = y_test[:subset_size_test]

# Convert y_test_subset to torch LongTensor
y_test_subset = torch.LongTensor(y_test_subset)

# Convert X_test_subset to list
X_test_list = list(X_test_subset['Tweet'])

# Create a smaller test Dataset
test_set_subset = Dataset.from_dict({
    'texts': X_test_list,
    'labels': y_test_subset
})

In [ ]:
# Map the preprocess function to your test dataset
test_set_tokenized = test_set.map(preprocess_function_test, batched=True)
# Define the class names
class_names = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']



Map:   0%|          | 0/7338 [00:00<?, ? examples/s]

<font color = 'indianred'>*Compute Metric for Test Set*</font>

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Check shapes
    print(f"logits shape: {logits[0].shape}")
    print(f"labels shape: {labels.shape}")

    # Flatten logits and labels
    logits = logits[0].reshape(-1, logits[0].shape[-1])
    labels = labels.reshape(-1, labels.shape[-1])

    # Convert logits to probabilities and then to predictions
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()
    predictions = (probabilities > 0.5).astype(int)

    # Flatten predictions
    predictions = predictions.flatten()
    labels = labels.flatten()

    # Compute metrics using sklearn
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    hamming_loss_metric = hamming_loss(labels, predictions)

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hamming_loss': hamming_loss_metric
    }


<font color = 'indianred'>*Training Arguments*</font>

In [ ]:
# Configure the evaluation parameters
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=128,
    do_train=False,
    do_eval=True,
    report_to=[],
    run_name=run_name
)

<font color = 'indianred'>*Instantiate Trainer*</font>

In [ ]:
# Initialize the Trainer for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_set_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


<font color = 'indianred'>*Evaluate using Trainer*</font>

In [ ]:
test_results= trainer.evaluate()

logits shape: (7338, 11)
labels shape: (7338, 11)


In [ ]:
test_results

{'eval_loss': nan,
 'eval_accuracy': 0.7855249138977676,
 'eval_f1': 0.4399406066997863,
 'eval_precision': 0.8927624569488838,
 'eval_recall': 0.5,
 'eval_hamming_loss': 0.21447508610223245,
 'eval_runtime': 55.7251,
 'eval_samples_per_second': 131.682,
 'eval_steps_per_second': 1.041}

In [ ]:
wandb.log({"test_accuracy": test_results["eval_accuracy"], "test_loss": test_results["eval_loss"], "test_f1": test_results["eval_f1"]})


In [ ]:
test_output = trainer.predict(test_set_tokenized)

logits shape: (7338, 11)
labels shape: (7338, 11)


In [ ]:
wandb.finish()

eval/accuracy,▁▁▁▁▁▁
eval/f1,▁▁▁▁▁▁
eval/hamming_loss,▁▁▁▁▁▁
eval/precision,▁▁▁▁▁▁
eval/recall,▁▁▁▁▁▁
eval/runtime,▁▃▃█▂▅
eval/samples_per_second,█▅▅▁▇▃
eval/steps_per_second,█▅▅▁▇▃
eval_accuracy,▁
eval_f1,▁
test_accuracy,▁


# <Font color = 'indianred'> **8. Model Inference**



In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
custom_pipeline = pipeline(
    task="text-classification",
    model=checkpoint,
    tokenizer=checkpoint,
    device=0)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/data/models/flan_T5_base_final/checkpoint-10 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<font color = 'indianred'> **Test the pipeline on a single example**

In [ ]:
sample = test_set['texts'][0]
preds = custom_pipeline(sample)
preds


[{'label': 'trust', 'score': 0.1485118716955185}]

<font color = 'indianred'> **Test the pipeline on a large dataset**

In [ ]:
predictions = custom_pipeline(test_set['texts'], truncation=True)

In [ ]:
predictions[0:10]


[{'label': 'trust', 'score': 0.1485118716955185},
 {'label': 'trust', 'score': 0.1465597301721573},
 {'label': 'trust', 'score': 0.14706994593143463},
 {'label': 'trust', 'score': 0.1446983963251114},
 {'label': 'trust', 'score': 0.15649524331092834},
 {'label': 'trust', 'score': 0.14792369306087494},
 {'label': 'trust', 'score': 0.142600417137146},
 {'label': 'trust', 'score': 0.1387442648410797},
 {'label': 'trust', 'score': 0.15280656516551971},
 {'label': 'trust', 'score': 0.13020241260528564}]